# Imports

In [1]:
import numpy as np
import scipy
import pandas as pd
import random
import math
import os
import sys
import shutil
import rasterio

import arcpy
from arcpy import env

from arcpy.sa import Raster
from arcpy.sa import *
arcpy.CheckOutExtension("Spatial")

'CheckedOut'

In [2]:
from arcpy.ia import *
arcpy.CheckOutExtension("ImageAnalyst")

'CheckedOut'

In [5]:
# # This should print the project folder
# os.chdir(os.path.dirname(os.getcwd()))
# arcpy.env.workspace = os.getcwd()
# os.getcwd()

In [ ]:
# define user folder
user_folder = <path to user folder>

In [ ]:
break

# Create shapefiles

## Svalbard & North Sea 2019 (with date)

This script basically combines different polygons of a same SAR scene to create one single shapefile. This step will be later required for running the create training samples script

In [ ]:
import geopandas as gpd
# gdf = gpd.read_file(r'C:\{user_folder}\.1\--Data\labeled_slick_polygons\s1ewhh_barents.geojson') # Use EPSG:32636
# gdf = gpd.read_file(r'C:\{user_folder}\.1\--Data\labeled_slick_polygons\s1iwvv_ns2019.geojson') # Use EPSG:32631
# gdf = gpd.read_file(r'C:\{user_folder}\.1\--Data\labeled_slick_polygons\s1iwvv_rune.geojson') # Use EPSG:32631
# gdf = gpd.read_file(r'C:\{user_folder}\.1\--Data\labeled_slick_polygons\s1ewhh_pkf.geojson') # Use EPSG:32632

In [3]:
# This drops columns which are not used later. Reduces data size and avoids raising warnings in the export shapefiles function below
gdf.drop(columns={'image_id','Shape_Length','Shape_Area','OBJECTID','OBJECTID_1','OBJECTID_12','layer'},inplace=True,errors='ignore')
gdf

,date,geometry
0,2015_08_27_144622.tiff,"POLYGON ((31.58613 75.20990, 31.58749 75.20834..."
1,2015_08_27_144622.tiff,"POLYGON ((31.82077 75.24779, 31.83398 75.24805..."
2,2015_07_03_145449.tiff,"POLYGON ((31.55871 75.20825, 31.57022 75.20004..."
3,2015_07_02_042905.tiff,"POLYGON ((31.71543 75.28333, 31.70599 75.28052..."
4,2015_07_02_042905.tiff,"POLYGON ((31.93318 75.24572, 31.93250 75.24452..."
...,...,...
278,2020_08_27_151138.tiff,"POLYGON ((31.70834 75.22585, 31.70614 75.22632..."
279,2020_08_27_151138.tiff,"POLYGON ((31.55645 75.22683, 31.55448 75.22878..."
280,2018_08_27_051729.tiff,"POLYGON ((31.84946 75.26372, 31.85241 75.26407..."
281,2018_08_27_051729.tiff,"POLYGON ((31.77864 75.30729, 31.78991 75.30976..."


In [24]:
# Fix for rune's format
# gdf['date'] = gdf['date'].dt.strftime('%Y_%m_%d_%H%M%S.tiff')
gdf

,date,geometry
0,2015_08_27_144622.tiff,"POLYGON ((31.58613 75.20990, 31.58749 75.20834..."
1,2015_08_27_144622.tiff,"POLYGON ((31.82077 75.24779, 31.83398 75.24805..."
2,2015_07_03_145449.tiff,"POLYGON ((31.55871 75.20825, 31.57022 75.20004..."
3,2015_07_13_052621.tiff,"POLYGON ((31.55831 75.20775, 31.57788 75.20572..."
4,2015_07_02_042905.tiff,"POLYGON ((31.71543 75.28333, 31.70599 75.28052..."
...,...,...
279,2020_08_27_151138.tiff,"POLYGON ((31.70834 75.22585, 31.70614 75.22632..."
280,2020_08_27_151138.tiff,"POLYGON ((31.55645 75.22683, 31.55448 75.22878..."
281,2018_08_27_051729.tif,"POLYGON ((31.84946 75.26372, 31.85241 75.26407..."
282,2018_08_27_051729.tif,"POLYGON ((31.77864 75.30729, 31.78991 75.30976..."


In [5]:
# We will reproject the gdf to the right crs depending on the surveyed area
# Barents: epsg:32636 ; North_sea: epsg:32631 ; PKF: epsg:32632 ; 

gdf = gdf.to_crs(epsg=32636)
gdf.head()

,date,geometry
0,2015_08_27_144622.tiff,"POLYGON ((459714.845 8347506.663, 459749.289 8..."
1,2015_08_27_144622.tiff,"POLYGON ((466483.552 8351587.914, 466859.381 8..."
2,2015_07_03_145449.tiff,"POLYGON ((458929.112 8347342.298, 459234.979 8..."
3,2015_07_02_042905.tiff,"POLYGON ((463575.914 8355614.526, 463301.488 8..."
4,2015_07_02_042905.tiff,"POLYGON ((469673.971 8351296.503, 469652.296 8..."


In [6]:
def export_grouped_shapefiles(gdf, out_path):
    # Ensure the out_path exists
    os.makedirs(out_path, exist_ok=True)
    
    # Group the GeoDataFrame by 'date' column
    grouped = gdf.groupby('date')

    # Iterate over each group
    for date, group in grouped:
        # Create a folder for each date
        date_folder = os.path.join(out_path, str(date))
        os.makedirs(date_folder, exist_ok=True)
        
        # Define the path to save the shapefile for each group
        shapefile_path = os.path.join(date_folder, f"{date.strip('.tiff')}.shp")
        
        # Export the group to the shapefile
        group.to_file(shapefile_path)
        
        print(f"Shapefile for {date} saved at {shapefile_path}")

In [ ]:
# #Runs the export grouped shapefiles 

# # shp_folder_path = r"C:\{user_folder}\.1\--Data\labeled_slick_polygons\s1_barents_shp"
# # shp_folder_path = r"C:\{user_folder}\.1\--Data\labeled_slick_polygons\s1_ns2019_shp"
# # shp_folder_path = r"C:\{user_folder}\.1\--Data\labeled_slick_polygons\s1_rune_shp"
# # shp_folder_path = r"C:\{user_folder}\.1\--Data\labeled_slick_polygons\s1_pkf_shp"
# export_grouped_shapefiles(gdf, shp_folder_path)

Organize the shapefiles in month subfolders. Not needed per se but helps track script progress and data management

In [8]:
# # Loop through the folders in the directory
# for folder_name in os.listdir(shp_folder_path):
#     # Construct full path of the folder
#     folder_path = os.path.join(shp_folder_path, folder_name)
    
#     # Ensure it's a folder
#     if os.path.isdir(folder_path):
#         # Get filenames
#         filename = os.path.basename(folder_name)
        
#         # Extract the month (characters at positions 5 and 6 represent the month) (format: yyyy_mm_dd_HHMMSS.tiff)
#         month = filename[5:7] 

#         # Convert month to an integer to remove leading zeros (e.g., '01' -> 1, '09' -> 9)
#         month_int = str(int(month))  # This converts '01' to '1', '02' to '2', etc.

#         # Create the new sub-folder based on the month (1 to 12)
#         new_folder_path = os.path.join(shp_folder_path, month_int)
        
#         # Create the sub-folder if it doesn't exist
#         if not os.path.exists(new_folder_path):
#             os.makedirs(new_folder_path)
        
#         # Move the folder into the respective sub-folder
#         new_folder_path_full = os.path.join(new_folder_path, folder_name)
        
#         # Move the folder
#         shutil.move(folder_path, new_folder_path_full)
#         print(f"Moved {folder_name} to {new_folder_path}")


# Create training samples

In [4]:
buffer = 8000

In [5]:
# Check for and disconnect all active connections to a geodatabase
arcpy.ClearWorkspaceCache_management()

<Result 'true'>

## PKF

In [ ]:
# Params
zone = 'pkf'
gdb_workspace = r"C:\{user_folder}\OneDrive - University of Bergen\ArcGIS\Projects\label_sar\label_sar.gdb" # replace with path to gdb
shp_folder_path = rf"C:\{user_folder}\.1\--Data\labeled_slick_polygons\s1_{zone}_shp"
tile_sizes = [256,512]
pixel_sizes = [10,25]

In [ ]:
## STRETCH NEW TEST 8bit WITTHOUT COPY RASTER

arcpy.env.workspace = gdb_workspace

# variable names for temp rasters
temp_raster_clipped = 'temp_raster_clipped'
temp_raster_8bit = 'temp_raster_8bit'
temp_paths = [temp_raster_clipped,temp_raster_8bit]

# load each shapefile polygon
for tif_name in os.listdir(shp_folder_path):
    f_name = tif_name.strip('.tiff')
    shp_path = os.path.join(shp_folder_path, f'{tif_name}/{f_name}.shp')
    
    # create samples for each pixel size (10, 25)
    for pixel_size in pixel_sizes:
        # check and delete temp rasters if they exist
        for p in temp_paths:
            if arcpy.Exists(p):
                try:
                    arcpy.management.Delete(p)
                    print(f"Deleted: {p}")
                except Exception as e:
                    print(f"Error deleting {p}: {e}")
            else:
                print(f"File does not exist: {p}")

        # define input raster path
        in_raster_path = rf"C:\{user_folder}\.1\--Data\s1\{zone}_reproj_{pixel_size}\{f_name}.tif"
        
        # check if both samples with (pixel size) already exists. if yes, skip
        if not (os.path.isdir(rf'C:\{user_folder}\.1\--Data\training_samples\ts_256_{pixel_size}\ts_{zone}\{f_name}') and 
                os.path.isdir(rf'C:\{user_folder}\.1\--Data\training_samples\ts_512_{pixel_size}\ts_{zone}\{f_name}')):
            
            # Normalization issue: Clip a bounding box around the slicks
            # get the bbox min max coordinates
            extent = arcpy.Describe(shp_path).extent

            # Extract the coordinates of the bounding box. Add a 5km buffer around
            min_x = extent.XMin - buffer
            min_y = extent.YMin - buffer
            max_x = extent.XMax + buffer
            max_y = extent.YMax + buffer
            with arcpy.EnvManager(pyramid="NONE"):
                arcpy.management.Clip(
                    in_raster=in_raster_path,
                    rectangle=f"{min_x} {min_y} {max_x} {max_y}",
                    out_raster=temp_raster_clipped,
                    # nodata_value="65536",
                    clipping_geometry="NONE",
                    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
                )
            print(f'clipped {tif_name}')

            # Apply standard deviation stretch (using 2 standard deviations)
            stretched_raster = Stretch(temp_raster_clipped, "StdDev", 2)

            # # convert raster to 8bit
            # arcpy.management.CopyRaster(
            #     stretched_raster,     
            #     temp_raster_8bit, 
            #     pixel_type="8_BIT_UNSIGNED" 
            # )
            # print(f'{tif_name} standardized and converted to bit')
            
        # create samples for each tile size (256, 512)
        for tile_size in tile_sizes:
            # define parameters for exporting samples
            out_path = rf'C:\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_{zone}\{f_name}'
            stride = tile_size / 2

            # create training samples for the tile size and the pixel size
            if not os.path.isdir(out_path):
                print(f'Creating training samples for {tif_name} ')
                ExportTrainingDataForDeepLearning(
                    in_raster=stretched_raster, 
                    out_folder=out_path,
                    in_class_data=shp_path,
                    image_chip_format='TIFF',
                    tile_size_x=f'{tile_size}',
                    tile_size_y=f'{tile_size}',
                    stride_x=stride,
                    stride_y=stride,
                    metadata_format='Classified_Tiles',
                    buffer_radius=0,
                    # rotation_angle=90,
                    # min_polygon_overlap_ratio=0.1,
                )
            print(f'created samples at {out_path}')

## Barents

In [ ]:
# Params
zone = 'barents'
gdb_workspace = r"C:\{user_folder}\Projects\label_sar\label_sar.gdb" # replace with path to gdb
shp_folder_path = rf"C:\{user_folder}\.1\--Data\labeled_slick_polygons\s1_{zone}_shp"
tile_sizes = [256,512]
pixel_sizes = [10,25]

In [ ]:
# ## STRETCH NEW TEST 8bit

# arcpy.env.workspace = gdb_workspace

# # variable names for temp rasters
# temp_raster_clipped = 'temp_raster_clipped'
# temp_raster_8bit = 'temp_raster_8bit'
# temp_paths = [temp_raster_clipped,temp_raster_8bit]

# # load each shapefile polygon
# for tif_name in os.listdir(shp_folder_path):
#     f_name = tif_name.strip('.tiff')
#     shp_path = os.path.join(shp_folder_path, f'{tif_name}/{f_name}.shp')
    
#     # create samples for each pixel size (10, 25)
#     for pixel_size in pixel_sizes:
#         # check and delete temp rasters if they exist
#         for p in temp_paths:
#             if arcpy.Exists(p):
#                 try:
#                     arcpy.management.Delete(p)
#                     print(f"Deleted: {p}")
#                 except Exception as e:
#                     print(f"Error deleting {p}: {e}")
#             else:
#                 print(f"File does not exist: {p}")

#         # define input raster path
#         in_raster_path = rf"C:\{user_folder}\.1\--Data\s1\{zone}_reproj_{pixel_size}\{f_name}.tif"
        
#         # check if both samples with (pixel size) already exists. if yes, skip
#         if not (os.path.isdir(rf'C:\{user_folder}\.1\--Data\training_samples\ts_256_{pixel_size}\ts_{zone}\{f_name}') and 
#                 os.path.isdir(rf'C:\{user_folder}\.1\--Data\training_samples\ts_512_{pixel_size}\ts_{zone}\{f_name}')):
            
#             # Normalization issue: Clip a bounding box around the slicks
#             # get the bbox min max coordinates
#             extent = arcpy.Describe(shp_path).extent

#             # Extract the coordinates of the bounding box. Add a 5km buffer around
#             min_x = extent.XMin - buffer
#             min_y = extent.YMin - buffer
#             max_x = extent.XMax + buffer
#             max_y = extent.YMax + buffer

#             with arcpy.EnvManager(pyramid="NONE"):
#                 arcpy.management.Clip(
#                     in_raster=in_raster_path,
#                     rectangle=f"{min_x} {min_y} {max_x} {max_y}",
#                     out_raster=temp_raster_clipped,
#                     # nodata_value="65536",
#                     clipping_geometry="NONE",
#                     maintain_clipping_extent="NO_MAINTAIN_EXTENT"
#                 )
#             print(f'clipped {tif_name}')

#             # Apply standard deviation stretch (using 2 standard deviations)
#             stretched_raster = Stretch(temp_raster_clipped, "StdDev", 2)

#             # # convert raster to 8bit
#             # arcpy.management.CopyRaster(
#             #     stretched_raster,     
#             #     temp_raster_8bit, 
#             #     pixel_type="8_BIT_UNSIGNED" 
#             # )
#             print(f'{tif_name} standardized and converted to bit')
            
#         # create samples for each tile size (256, 512)
#         for tile_size in tile_sizes:
#             # define parameters for exporting samples
#             out_path = rf'C:\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_{zone}\{f_name}'
#             stride = tile_size / 2

#             # create training samples for the tile size and the pixel size
#             if not os.path.isdir(out_path):
#                 print(f'Creating training samples for {tif_name} ')
#                 ExportTrainingDataForDeepLearning(
#                     in_raster=stretched_raster, 
#                     out_folder=out_path,
#                     in_class_data=shp_path,
#                     image_chip_format='TIFF',
#                     tile_size_x=f'{tile_size}',
#                     tile_size_y=f'{tile_size}',
#                     stride_x=stride,
#                     stride_y=stride,
#                     metadata_format='Classified_Tiles',
#                     buffer_radius=0,
#                     # rotation_angle=90,
#                     # min_polygon_overlap_ratio=0.1,
#                 )
#             print(f'created samples at {out_path}')

## North sea 2019

In [ ]:
# # Params
# zone = 'ns2019'
# gdb_workspace = r"C:\{user_folder}\OneDrive - University of Bergen\ArcGIS\Projects\label_sar\label_sar.gdb" # replace with path to gdb
# shp_folder_path = rf"C:\{user_folder}\.1\--Data\labeled_slick_polygons\s1_{zone}_shp"
# tile_sizes = [256,512]
# pixel_sizes = [10,25]

In [ ]:
# ## STRETCH NEW TEST 8bit

# arcpy.env.workspace = gdb_workspace

# # variable names for temp rasters
# temp_raster_clipped = 'temp_raster_clipped'
# temp_raster_8bit = 'temp_raster_8bit'
# temp_paths = [temp_raster_clipped,temp_raster_8bit]

# # load each shapefile polygon
# for tif_name in os.listdir(shp_folder_path):
#     f_name = tif_name.strip('.tiff')
#     shp_path = os.path.join(shp_folder_path, f'{tif_name}/{f_name}.shp')
    
#     # create samples for each pixel size (10, 25)
#     for pixel_size in pixel_sizes:
#         # check and delete temp rasters if they exist
#         for p in temp_paths:
#             if arcpy.Exists(p):
#                 try:
#                     arcpy.management.Delete(p)
#                     print(f"Deleted: {p}")
#                 except Exception as e:
#                     print(f"Error deleting {p}: {e}")
#             else:
#                 print(f"File does not exist: {p}")

#         # define input raster path
#         in_raster_path = rf"C:\{user_folder}\.1\--Data\s1\{zone}_reproj_{pixel_size}\{f_name}.tif"
        
#         # check if both samples with (pixel size) already exists. if yes, skip
#         if not (os.path.isdir(rf'C:\{user_folder}\.1\--Data\training_samples\ts_256_{pixel_size}\ts_{zone}\{f_name}') and 
#                 os.path.isdir(rf'C:\{user_folder}\.1\--Data\training_samples\ts_512_{pixel_size}\ts_{zone}\{f_name}')):
            
#             # Normalization issue: Clip a bounding box around the slicks
#             # get the bbox min max coordinates
#             extent = arcpy.Describe(shp_path).extent

#             # Extract the coordinates of the bounding box. Add a 5km buffer around
#             min_x = extent.XMin - buffer
#             min_y = extent.YMin - buffer
#             max_x = extent.XMax + buffer
#             max_y = extent.YMax + buffer
#             with arcpy.EnvManager(pyramid="NONE"):
#                 arcpy.management.Clip(
#                     in_raster=in_raster_path,
#                     rectangle=f"{min_x} {min_y} {max_x} {max_y}",
#                     out_raster=temp_raster_clipped,
#                     # nodata_value="65536",
#                     clipping_geometry="NONE",
#                     maintain_clipping_extent="NO_MAINTAIN_EXTENT"
#                 )
#             print(f'clipped {tif_name}')

#             # Apply standard deviation stretch (using 2 standard deviations)
#             stretched_raster = Stretch(temp_raster_clipped, "StdDev", 2)

#             # # convert raster to 8bit
#             # arcpy.management.CopyRaster(
#             #     stretched_raster,     
#             #     temp_raster_8bit, 
#             #     pixel_type="8_BIT_UNSIGNED" 
#             # )
#             # print(f'{tif_name} standardized and converted to bit')
            
#         # create samples for each tile size (256, 512)
#         for tile_size in tile_sizes:
#             # define parameters for exporting samples
#             out_path = rf'C:\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_{zone}\{f_name}'
#             stride = tile_size / 2

#             # create training samples for the tile size and the pixel size
#             if not os.path.isdir(out_path):
#                 print(f'Creating training samples for {tif_name} ')
#                 ExportTrainingDataForDeepLearning(
#                     in_raster=stretched_raster, 
#                     out_folder=out_path,
#                     in_class_data=shp_path,
#                     image_chip_format='TIFF',
#                     tile_size_x=f'{tile_size}',
#                     tile_size_y=f'{tile_size}',
#                     stride_x=stride,
#                     stride_y=stride,
#                     metadata_format='Classified_Tiles',
#                     buffer_radius=0,
#                     # rotation_angle=90,
#                     # min_polygon_overlap_ratio=0.1,
#                 )
#             print(f'created samples at {out_path}')

## North sea Rune

In [ ]:
# Params
zone = 'rune'
gdb_workspace = r"C:\{user_folder}\OneDrive - University of Bergen\ArcGIS\Projects\label_sar\label_sar.gdb" # replace with path to gdb
shp_folder_path = rf"C:\{user_folder}\.1\--Data\labeled_slick_polygons\s1_{zone}_shp"
tile_sizes = [256,512]
pixel_sizes = [10,25]

In [ ]:
# ## STRETCH NEW TEST 8bit

# arcpy.env.workspace = gdb_workspace

# # variable names for temp rasters
# temp_raster_clipped = 'temp_raster_clipped'
# temp_raster_8bit = 'temp_raster_8bit'
# temp_paths = [temp_raster_clipped,temp_raster_8bit]

# # load each shapefile polygon
# for tif_name in os.listdir(shp_folder_path):
#     f_name = tif_name.strip('.tiff')
#     shp_path = os.path.join(shp_folder_path, f'{tif_name}/{f_name}.shp')
    
#     # create samples for each pixel size (10, 25)
#     for pixel_size in pixel_sizes:
#         # check and delete temp rasters if they exist
#         for p in temp_paths:
#             if arcpy.Exists(p):
#                 try:
#                     arcpy.management.Delete(p)
#                     print(f"Deleted: {p}")
#                 except Exception as e:
#                     print(f"Error deleting {p}: {e}")
#             else:
#                 print(f"File does not exist: {p}")

#         # define input raster path
#         in_raster_path = rf"C:\Users\{user_folder}\.1\--Data\s1\{zone}_reproj_{pixel_size}\{f_name}.tif"
        
#         # check if both samples with (pixel size) already exists. if yes, skip
#         if not (os.path.isdir(rf'C:\{user_folder}\.1\--Data\training_samples\ts_256_{pixel_size}\ts_{zone}\{f_name}') and 
#                 os.path.isdir(rf'C:\{user_folder}\.1\--Data\training_samples\ts_512_{pixel_size}\ts_{zone}\{f_name}')):
            
#             # Normalization issue: Clip a bounding box around the slicks
#             # get the bbox min max coordinates
#             extent = arcpy.Describe(shp_path).extent

#             # Extract the coordinates of the bounding box. Add a 5km buffer around
#             min_x = extent.XMin - buffer
#             min_y = extent.YMin - buffer
#             max_x = extent.XMax + buffer
#             max_y = extent.YMax + buffer

#             with arcpy.EnvManager(pyramid="NONE"):
#                 arcpy.management.Clip(
#                     in_raster=in_raster_path,
#                     rectangle=f"{min_x} {min_y} {max_x} {max_y}",
#                     out_raster=temp_raster_clipped,
#                     # nodata_value="65536",
#                     clipping_geometry="NONE",
#                     maintain_clipping_extent="NO_MAINTAIN_EXTENT"
#                 )
#             print(f'clipped {tif_name}')

#             # Apply standard deviation stretch (using 2 standard deviations)
#             stretched_raster = Stretch(temp_raster_clipped, "StdDev", 2)

#             # # convert raster to 8bit
#             # arcpy.management.CopyRaster(
#             #     stretched_raster,     
#             #     temp_raster_8bit, 
#             #     pixel_type="8_BIT_UNSIGNED" 
#             # )
#             print(f'{tif_name} standardized and converted to bit')
            
#         # create samples for each tile size (256, 512)
#         for tile_size in tile_sizes:
#             # define parameters for exporting samples
#             out_path = rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_{zone}\{f_name}'
#             stride = tile_size / 2

#             # create training samples for the tile size and the pixel size
#             if not os.path.isdir(out_path):
#                 print(f'Creating training samples for {tif_name} ')
#                 ExportTrainingDataForDeepLearning(
#                     in_raster=stretched_raster, 
#                     out_folder=out_path,
#                     in_class_data=shp_path,
#                     image_chip_format='TIFF',
#                     tile_size_x=f'{tile_size}',
#                     tile_size_y=f'{tile_size}',
#                     stride_x=stride,
#                     stride_y=stride,
#                     metadata_format='Classified_Tiles',
#                     buffer_radius=0,
#                     # rotation_angle=90,
#                     # min_polygon_overlap_ratio=0.1,
#                 )
#             print(f'created samples at {out_path}')

In [ ]:
# # from arcpy import env
# # arcpy.env.parallelProcessingFactor = "50%"

# arcpy.env.workspace = gdb_workspace

# # variable names for temp rasters
# temp_raster_clipped = 'temp_raster_clipped'
# temp_raster_8bit = 'temp_raster_8bit'
# temp_raster_64bit = 'temp_raster_64bit'
# temp_paths = [temp_raster_clipped,temp_raster_8bit, temp_raster_64bit]

# # load each shapefile polygon
# for tif_name in os.listdir(shp_folder_path): 
#     f_name = tif_name.strip('.tiff')
#     shp_path = os.path.join(shp_folder_path, f'{tif_name}/{f_name}.shp')
    
#     # create samples for each pixel size (10, 25)
#     for pixel_size in pixel_sizes:
#         # check and delete temp rasters if they exist
#         for p in temp_paths:
#             if arcpy.Exists(p):
#                 try:
#                     arcpy.management.Delete(p)
#                     print(f"Deleted: {p}")
#                 except Exception as e:
#                     print(f"Error deleting {p}: {e}")
#             else:
#                 print(f"File does not exist: {p}")

#         # define input raster path
#         in_raster_path = rf"C:\Users\{user_folder}\.1\--Data\s1\{zone}_reproj_{pixel_size}\{f_name}.tif"
        
#         # check if both samples with (pixel size) already exists. if yes, skip
#         if not (os.path.isdir(rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_256_{pixel_size}\ts_{zone}\{f_name}') and 
#                 os.path.isdir(rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_512_{pixel_size}\ts_{zone}\{f_name}')):
            
#             # Normalization issue: Clip a bounding box around the slicks
#             # get the bbox min max coordinates
#             extent = arcpy.Describe(shp_path).extent

#             # Extract the coordinates of the bounding box. Add a 5km buffer around
#             min_x = extent.XMin - 5000
#             min_y = extent.YMin - 5000
#             max_x = extent.XMax + 5000
#             max_y = extent.YMax + 5000

#             with arcpy.EnvManager(pyramid="NONE"):
#                 arcpy.management.Clip(
#                     in_raster=in_raster_path,
#                     rectangle=f"{min_x} {min_y} {max_x} {max_y}",
#                     out_raster=temp_raster_clipped,
#                     # nodata_value="65536",
#                     clipping_geometry="NONE",
#                     maintain_clipping_extent="NO_MAINTAIN_EXTENT"
#                 )
#             print(f'clipped raster {temp_raster_clipped}')
#             # Get min and max values from the statistics property
#             min_val = arcpy.GetRasterProperties_management(temp_raster_clipped, "MINIMUM")
#             max_val = arcpy.GetRasterProperties_management(temp_raster_clipped, "MAXIMUM")

#             # min-max normalize 0-255
#             normalized = RasterCalculator(
#                 rasters = [temp_raster_clipped],
#                 input_names = 'x', 
#                 expression = f'(((x - {min_val}) / ({max_val} - {min_val})) * 255)'
#                 )
#             # save the temporary raster
#             normalized.save(temp_raster_64bit) # cannot overwrite
#             print(f'normalized {temp_raster_64bit}')

#             # convert raster to 8bit
#             arcpy.management.CopyRaster(
#                 temp_raster_64bit,     
#                 temp_raster_8bit, 
#                 pixel_type="8_BIT_UNSIGNED" 
#             )

#         # create samples for each tile size (256, 512)
#         for tile_size in tile_sizes:
#             # define parameters for exporting samples
#             out_path = rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_{zone}\{f_name}'
#             stride = tile_size / 2

#             # create training samples for the tile size and the pixel size
#             if not os.path.isdir(out_path):
#                 print(f'Creating training samples with {temp_raster_8bit} at {out_path} ')
#                 ExportTrainingDataForDeepLearning(
#                     in_raster=temp_raster_8bit, 
#                     out_folder=out_path,
#                     in_class_data=shp_path,
#                     image_chip_format='TIFF',
#                     tile_size_x=f'{tile_size}',
#                     tile_size_y=f'{tile_size}',
#                     stride_x=stride,
#                     stride_y=stride,
#                     metadata_format='Classified_Tiles',
#                     buffer_radius=0,
#                     # rotation_angle=90,
#                     # min_polygon_overlap_ratio=0.1,
#                 )
#             print(f'created samples at {out_path}')

In [6]:
# break

# Old create samples

### PKF

In [ ]:
# # from arcpy import env
# # arcpy.env.parallelProcessingFactor = "50%"

# # Loop through each folder
# for folder in os.listdir(r"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_pkf_shp"):
#     # Define the path to the shp folders
#     month_folder = rf"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_pkf_shp\{folder}"

#     # Find shapefile and tif files paths
#     for tif_name in os.listdir(month_folder):
#         f_name = tif_name.strip('.tiff')
#         shp_path = os.path.join(month_folder, f'{tif_name}/{f_name}.shp')
#         # print(shp_path)

#         raster_path = rf"C:\Users\{user_folder}\.1\--Data\s1\pkf_reproj_{pixel_size}\{f_name}.tif"
#         temp_8bit_path = r'C:\Users\{user_folder}\OneDrive - University of Bergen\ArcGIS\Projects\label_sar\label_sar.gdb\TempRaster'
#         out_path = rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_pkf\{f_name}'
#         stride = tile_size / 2
#         print(f'Processing: {out_path}')

#         try:
#         # arcpy.env.workspace = r'OK'
#         ##Copy File RasterDataset to GDB Dataset with Background and Nodata setting
#             arcpy.management.CopyRaster(
#                 in_raster = raster_path,
#                 out_rasterdataset = temp_8bit_path,
#                 # nodata_value = '0',
#                 pixel_type = '8_BIT_UNSIGNED',
#                 scale_pixel_value = 'NONE'
#                 )
#         except:
#             print("Copy Raster example failed.")
#             print(arcpy.GetMessages())
    
#         # Execute the ExportTrainingDataForDeepLearning tool
#         print('exporting training samples...')
#         ExportTrainingDataForDeepLearning(
#             in_raster=temp_8bit_path, #temp_raster_path
#             out_folder=out_path,
#             in_class_data=shp_path,
#             image_chip_format='TIFF',
#             tile_size_x=f'{tile_size}',
#             tile_size_y=f'{tile_size}',
#             stride_x=stride,
#             stride_y=stride,
#             metadata_format='Classified_Tiles',
#             buffer_radius=0,
#             # rotation_angle=90,
#             min_polygon_overlap_ratio=0.15, #5min47 without this #538 with
#         )
#         # Delete temp 8bit raster
#         arcpy.management.Delete(temp_8bit_path)
        
#     print(f'Processed month {folder}') 

### Barents

In [ ]:
# # from arcpy import env
# # arcpy.env.parallelProcessingFactor = "50%"

# # Loop through each folder
# for folder in os.listdir(r"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_barents_shp"):
#     # Define the path to the shp folders
#     month_folder = rf"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_barents_shp\{folder}"

#     # Find shapefile and tif files paths
#     for tif_name in os.listdir(month_folder):
#         f_name = tif_name.strip('.tiff')
#         shp_path = os.path.join(month_folder, f'{tif_name}/{f_name}.shp')
#         # print(shp_path)

#         raster_path = rf"C:\Users\{user_folder}\.1\--Data\s1\barents_reproj_{pixel_size}\{f_name}.tif"
#         temp_8bit_path = r'C:\Users\{user_folder}\OneDrive - University of Bergen\ArcGIS\Projects\label_sar\label_sar.gdb\TempRaster'
#         out_path = rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_barents\{f_name}'
#         stride = tile_size / 2
#         print(f'Processing: {out_path}')

#         try:
#         # arcpy.env.workspace = r'OK'
#         ##Copy File RasterDataset to GDB Dataset with Background and Nodata setting
#             arcpy.management.CopyRaster(
#                 in_raster = raster_path,
#                 out_rasterdataset = temp_8bit_path,
#                 # nodata_value = '0',
#                 pixel_type = '8_BIT_UNSIGNED',
#                 scale_pixel_value = 'NONE'
#                 )
#         except:
#             print("Copy Raster example failed.")
#             print(arcpy.GetMessages())
    
#         # Execute the ExportTrainingDataForDeepLearning tool
#         print('exporting training samples...')
#         ExportTrainingDataForDeepLearning(
#             in_raster=temp_8bit_path, #temp_raster_path
#             out_folder=out_path,
#             in_class_data=shp_path,
#             image_chip_format='TIFF',
#             tile_size_x=f'{tile_size}',
#             tile_size_y=f'{tile_size}',
#             stride_x=stride,
#             stride_y=stride,
#             metadata_format='Classified_Tiles',
#             buffer_radius=0,
#             # rotation_angle=90,
#             min_polygon_overlap_ratio=0.15, #5min47 without this #538 with
#         )
#         # Delete temp 8bit raster
#         arcpy.management.Delete(temp_8bit_path)
        
#     print(f'Processed month {folder}') 

### Rune North Sea

In [ ]:
# # from arcpy import env
# # arcpy.env.parallelProcessingFactor = "50%"

# # Loop through each folder
# for folder in os.listdir(r"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_rune_shp"):
#     # Define the path to the shp folders
#     month_folder = rf"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_rune_shp\{folder}"

#     # Find shapefile and tif files paths
#     for tif_name in os.listdir(month_folder):
#         f_name = tif_name.strip('.tiff')
#         shp_path = os.path.join(month_folder, f'{tif_name}/{f_name}.shp')
#         # print(shp_path)

#         raster_path = rf"C:\Users\{user_folder}\.1\--Data\s1\rune_reproj_{pixel_size}\{f_name}.tif"
#         temp_8bit_path = r'C:\Users\{user_folder}\OneDrive - University of Bergen\ArcGIS\Projects\label_sar\label_sar.gdb\TempRaster'
#         out_path = rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_rune\{f_name}'
#         stride = tile_size / 2
#         print(f'Processing: {out_path}')

#         try:
#         # arcpy.env.workspace = r'OK'
#         ##Copy File RasterDataset to GDB Dataset with Background and Nodata setting
#             arcpy.management.CopyRaster(
#                 in_raster = raster_path,
#                 out_rasterdataset = temp_8bit_path,
#                 # nodata_value = '0',
#                 pixel_type = '8_BIT_UNSIGNED',
#                 scale_pixel_value = 'NONE'
#                 )
#         except:
#             print("Copy Raster example failed.")
#             print(arcpy.GetMessages())
    
#         # Execute the ExportTrainingDataForDeepLearning tool
#         print('exporting training samples...')
#         ExportTrainingDataForDeepLearning(
#             in_raster=temp_8bit_path, #temp_raster_path
#             out_folder=out_path,
#             in_class_data=shp_path,
#             image_chip_format='TIFF',
#             tile_size_x=f'{tile_size}',
#             tile_size_y=f'{tile_size}',
#             stride_x=stride,
#             stride_y=stride,
#             metadata_format='Classified_Tiles',
#             buffer_radius=0,
#             # rotation_angle=90,
#             min_polygon_overlap_ratio=0.15, #5min47 without this #538 with
#         )
#         # Delete temp 8bit raster
#         arcpy.management.Delete(temp_8bit_path)
        
#     print(f'Processed month {folder}') 

### North Sea 2019

In [ ]:
# # from arcpy import env
# # arcpy.env.parallelProcessingFactor = "50%"

# # Loop through each folder
# for folder in os.listdir(r"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_ns2019_shp"):
#     # Define the path to the shp folders
#     month_folder = rf"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_ns2019_shp\{folder}"

#     # Find shapefile and tif files paths
#     for tif_name in os.listdir(month_folder):
#         f_name = tif_name.strip('.tiff')
#         shp_path = os.path.join(month_folder, f'{tif_name}/{f_name}.shp')
#         # print(shp_path)

#         raster_path = rf"C:\Users\{user_folder}\.1\--Data\s1\ns2019_reproj_{pixel_size}\{f_name}.tif"
#         temp_8bit_path = r'C:\Users\{user_folder}\OneDrive - University of Bergen\ArcGIS\Projects\label_sar\label_sar.gdb\TempRaster'
#         out_path = rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_ns2019\{f_name}'
#         stride = tile_size / 2
#         print(f'Processing: {out_path}')

#         try:
#         # arcpy.env.workspace = r'OK'
#         ##Copy File RasterDataset to GDB Dataset with Background and Nodata setting
#             arcpy.management.CopyRaster(
#                 in_raster = raster_path,
#                 out_rasterdataset = temp_8bit_path,
#                 # nodata_value = '0',
#                 pixel_type = '8_BIT_UNSIGNED',
#                 scale_pixel_value = 'NONE'
#                 )
#         except:
#             print("Copy Raster example failed.")
#             print(arcpy.GetMessages())
    
#         # Execute the ExportTrainingDataForDeepLearning tool
#         print('exporting training samples...')
#         ExportTrainingDataForDeepLearning(
#             in_raster=temp_8bit_path, #temp_raster_path
#             out_folder=out_path,
#             in_class_data=shp_path,
#             image_chip_format='TIFF',
#             tile_size_x=f'{tile_size}',
#             tile_size_y=f'{tile_size}',
#             stride_x=stride,
#             stride_y=stride,
#             metadata_format='Classified_Tiles',
#             buffer_radius=0,
#             # rotation_angle=90,
#             min_polygon_overlap_ratio=0.15, #5min47 without this #538 with
#         )
#         # Delete temp 8bit raster
#         arcpy.management.Delete(temp_8bit_path)
        
#     print(f'Processed month {folder}') 

## 256 x 10

In [ ]:
tile_size = 256
pixel_size = 10

### PKF

In [ ]:
# # from arcpy import env
# # arcpy.env.parallelProcessingFactor = "50%"

# # Loop through each folder
# for folder in os.listdir(r"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_pkf_shp"):
#     # Define the path to the shp folders
#     month_folder = rf"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_pkf_shp\{folder}"

#     # Find shapefile and tif files paths
#     for tif_name in os.listdir(month_folder):
#         f_name = tif_name.strip('.tiff')
#         shp_path = os.path.join(month_folder, f'{tif_name}/{f_name}.shp')
#         # print(shp_path)

#         raster_path = rf"C:\Users\{user_folder}\.1\--Data\s1\pkf_reproj_{pixel_size}\{f_name}.tif"
#         temp_8bit_path = r'C:\Users\{user_folder}\OneDrive - University of Bergen\ArcGIS\Projects\label_sar\label_sar.gdb\TempRaster'
#         out_path = rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_pkf\{f_name}'
#         stride = tile_size / 2
#         print(f'Processing: {out_path}')

#         try:
#         # arcpy.env.workspace = r'OK'
#         ##Copy File RasterDataset to GDB Dataset with Background and Nodata setting
#             arcpy.management.CopyRaster(
#                 in_raster = raster_path,
#                 out_rasterdataset = temp_8bit_path,
#                 # nodata_value = '0',
#                 pixel_type = '8_BIT_UNSIGNED',
#                 scale_pixel_value = 'NONE'
#                 )
#         except:
#             print("Copy Raster example failed.")
#             print(arcpy.GetMessages())
    
#         # Execute the ExportTrainingDataForDeepLearning tool
#         print('exporting training samples...')
#         ExportTrainingDataForDeepLearning(
#             in_raster=temp_8bit_path, #temp_raster_path
#             out_folder=out_path,
#             in_class_data=shp_path,
#             image_chip_format='TIFF',
#             tile_size_x=f'{tile_size}',
#             tile_size_y=f'{tile_size}',
#             stride_x=stride,
#             stride_y=stride,
#             metadata_format='Classified_Tiles',
#             buffer_radius=0,
#             # rotation_angle=90,
#             min_polygon_overlap_ratio=0.15, #5min47 without this #538 with
#         )
#         # Delete temp 8bit raster
#         arcpy.management.Delete(temp_8bit_path)
        
#     print(f'Processed month {folder}') 

### Barents

In [ ]:
# # from arcpy import env
# # arcpy.env.parallelProcessingFactor = "50%"

# # Loop through each folder
# for folder in os.listdir(r"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_barents_shp"):
#     # Define the path to the shp folders
#     month_folder = rf"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_barents_shp\{folder}"

#     # Find shapefile and tif files paths
#     for tif_name in os.listdir(month_folder):
#         f_name = tif_name.strip('.tiff')
#         shp_path = os.path.join(month_folder, f'{tif_name}/{f_name}.shp')
#         # print(shp_path)

#         raster_path = rf"C:\Users\{user_folder}\.1\--Data\s1\barents_reproj_{pixel_size}\{f_name}.tif"
#         temp_8bit_path = r'C:\Users\{user_folder}\OneDrive - University of Bergen\ArcGIS\Projects\label_sar\label_sar.gdb\TempRaster'
#         out_path = rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_barents\{f_name}'
#         stride = tile_size / 2
#         print(f'Processing: {out_path}')

#         try:
#         # arcpy.env.workspace = r'OK'
#         ##Copy File RasterDataset to GDB Dataset with Background and Nodata setting
#             arcpy.management.CopyRaster(
#                 in_raster = raster_path,
#                 out_rasterdataset = temp_8bit_path,
#                 # nodata_value = '0',
#                 pixel_type = '8_BIT_UNSIGNED',
#                 scale_pixel_value = 'NONE'
#                 )
#         except:
#             print("Copy Raster example failed.")
#             print(arcpy.GetMessages())
    
#         # Execute the ExportTrainingDataForDeepLearning tool
#         print('exporting training samples...')
#         ExportTrainingDataForDeepLearning(
#             in_raster=temp_8bit_path, #temp_raster_path
#             out_folder=out_path,
#             in_class_data=shp_path,
#             image_chip_format='TIFF',
#             tile_size_x=f'{tile_size}',
#             tile_size_y=f'{tile_size}',
#             stride_x=stride,
#             stride_y=stride,
#             metadata_format='Classified_Tiles',
#             buffer_radius=0,
#             # rotation_angle=90,
#             min_polygon_overlap_ratio=0.15, #5min47 without this #538 with
#         )
#         # Delete temp 8bit raster
#         arcpy.management.Delete(temp_8bit_path)
        
#     print(f'Processed month {folder}') 

### Rune North Sea

In [ ]:
# # from arcpy import env
# # arcpy.env.parallelProcessingFactor = "50%"

# # Loop through each folder
# for folder in os.listdir(r"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_rune_shp"):
#     # Define the path to the shp folders
#     month_folder = rf"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_rune_shp\{folder}"

#     # Find shapefile and tif files paths
#     for tif_name in os.listdir(month_folder):
#         f_name = tif_name.strip('.tiff')
#         shp_path = os.path.join(month_folder, f'{tif_name}/{f_name}.shp')
#         # print(shp_path)

#         raster_path = rf"C:\Users\{user_folder}\.1\--Data\s1\rune_reproj_{pixel_size}\{f_name}.tif"
#         temp_8bit_path = r'C:\Users\{user_folder}\OneDrive - University of Bergen\ArcGIS\Projects\label_sar\label_sar.gdb\TempRaster'
#         out_path = rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_rune\{f_name}'
#         stride = tile_size / 2
#         print(f'Processing: {out_path}')

#         try:
#         # arcpy.env.workspace = r'OK'
#         ##Copy File RasterDataset to GDB Dataset with Background and Nodata setting
#             arcpy.management.CopyRaster(
#                 in_raster = raster_path,
#                 out_rasterdataset = temp_8bit_path,
#                 # nodata_value = '0',
#                 pixel_type = '8_BIT_UNSIGNED',
#                 scale_pixel_value = 'NONE'
#                 )
#         except:
#             print("Copy Raster example failed.")
#             print(arcpy.GetMessages())
    
#         # Execute the ExportTrainingDataForDeepLearning tool
#         print('exporting training samples...')
#         ExportTrainingDataForDeepLearning(
#             in_raster=temp_8bit_path, #temp_raster_path
#             out_folder=out_path,
#             in_class_data=shp_path,
#             image_chip_format='TIFF',
#             tile_size_x=f'{tile_size}',
#             tile_size_y=f'{tile_size}',
#             stride_x=stride,
#             stride_y=stride,
#             metadata_format='Classified_Tiles',
#             buffer_radius=0,
#             # rotation_angle=90,
#             min_polygon_overlap_ratio=0.15, #5min47 without this #538 with
#         )
#         # Delete temp 8bit raster
#         arcpy.management.Delete(temp_8bit_path)
        
#     print(f'Processed month {folder}') 

### North Sea 2019

In [ ]:
# # from arcpy import env
# # arcpy.env.parallelProcessingFactor = "50%"

# # Loop through each folder
# for folder in os.listdir(r"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_ns2019_shp"):
#     # Define the path to the shp folders
#     month_folder = rf"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_ns2019_shp\{folder}"

#     # Find shapefile and tif files paths
#     for tif_name in os.listdir(month_folder):
#         f_name = tif_name.strip('.tiff')
#         shp_path = os.path.join(month_folder, f'{tif_name}/{f_name}.shp')
#         # print(shp_path)

#         raster_path = rf"C:\Users\{user_folder}\.1\--Data\s1\ns2019_reproj_{pixel_size}\{f_name}.tif"
#         temp_8bit_path = r'C:\Users\{user_folder}\OneDrive - University of Bergen\ArcGIS\Projects\label_sar\label_sar.gdb\TempRaster'
#         out_path = rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_ns2019\{f_name}'
#         stride = tile_size / 2
#         print(f'Processing: {out_path}')

#         try:
#         # arcpy.env.workspace = r'OK'
#         ##Copy File RasterDataset to GDB Dataset with Background and Nodata setting
#             arcpy.management.CopyRaster(
#                 in_raster = raster_path,
#                 out_rasterdataset = temp_8bit_path,
#                 # nodata_value = '0',
#                 pixel_type = '8_BIT_UNSIGNED',
#                 scale_pixel_value = 'NONE'
#                 )
#         except:
#             print("Copy Raster example failed.")
#             print(arcpy.GetMessages())
    
#         # Execute the ExportTrainingDataForDeepLearning tool
#         print('exporting training samples...')
#         ExportTrainingDataForDeepLearning(
#             in_raster=temp_8bit_path, #temp_raster_path
#             out_folder=out_path,
#             in_class_data=shp_path,
#             image_chip_format='TIFF',
#             tile_size_x=f'{tile_size}',
#             tile_size_y=f'{tile_size}',
#             stride_x=stride,
#             stride_y=stride,
#             metadata_format='Classified_Tiles',
#             buffer_radius=0,
#             # rotation_angle=90,
#             min_polygon_overlap_ratio=0.15, #5min47 without this #538 with
#         )
#         # Delete temp 8bit raster
#         arcpy.management.Delete(temp_8bit_path)
        
#     print(f'Processed month {folder}') 

## 512 x 25

In [ ]:
tile_size = 512
pixel_size = 25

### PKF

In [ ]:
# # from arcpy import env
# # arcpy.env.parallelProcessingFactor = "50%"

# # Loop through each folder
# for folder in os.listdir(r"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_pkf_shp"):
#     # Define the path to the shp folders
#     month_folder = rf"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_pkf_shp\{folder}"

#     # Find shapefile and tif files paths
#     for tif_name in os.listdir(month_folder):
#         f_name = tif_name.strip('.tiff')
#         shp_path = os.path.join(month_folder, f'{tif_name}/{f_name}.shp')
#         # print(shp_path)

#         raster_path = rf"C:\Users\{user_folder}\.1\--Data\s1\pkf_reproj_{pixel_size}\{f_name}.tif"
#         temp_8bit_path = r'C:\Users\{user_folder}\OneDrive - University of Bergen\ArcGIS\Projects\label_sar\label_sar.gdb\TempRaster'
#         out_path = rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_pkf\{f_name}'
#         stride = tile_size / 2
#         print(f'Processing: {out_path}')

#         try:
#         # arcpy.env.workspace = r'OK'
#         ##Copy File RasterDataset to GDB Dataset with Background and Nodata setting
#             arcpy.management.CopyRaster(
#                 in_raster = raster_path,
#                 out_rasterdataset = temp_8bit_path,
#                 # nodata_value = '0',
#                 pixel_type = '8_BIT_UNSIGNED',
#                 scale_pixel_value = 'NONE'
#                 )
#         except:
#             print("Copy Raster example failed.")
#             print(arcpy.GetMessages())
    
#         # Execute the ExportTrainingDataForDeepLearning tool
#         print('exporting training samples...')
#         ExportTrainingDataForDeepLearning(
#             in_raster=temp_8bit_path, #temp_raster_path
#             out_folder=out_path,
#             in_class_data=shp_path,
#             image_chip_format='TIFF',
#             tile_size_x=f'{tile_size}',
#             tile_size_y=f'{tile_size}',
#             stride_x=stride,
#             stride_y=stride,
#             metadata_format='Classified_Tiles',
#             buffer_radius=0,
#             # rotation_angle=90,
#             min_polygon_overlap_ratio=0.15, #5min47 without this #538 with
#         )
#         # Delete temp 8bit raster
#         arcpy.management.Delete(temp_8bit_path)
        
#     print(f'Processed month {folder}') 

### Barents

In [ ]:
# # from arcpy import env
# # arcpy.env.parallelProcessingFactor = "50%"

# # Loop through each folder
# for folder in os.listdir(r"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_barents_shp"):
#     # Define the path to the shp folders
#     month_folder = rf"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_barents_shp\{folder}"

#     # Find shapefile and tif files paths
#     for tif_name in os.listdir(month_folder):
#         f_name = tif_name.strip('.tiff')
#         shp_path = os.path.join(month_folder, f'{tif_name}/{f_name}.shp')
#         # print(shp_path)

#         raster_path = rf"C:\Users\{user_folder}\.1\--Data\s1\barents_reproj_{pixel_size}\{f_name}.tif"
#         temp_8bit_path = r'C:\Users\{user_folder}\OneDrive - University of Bergen\ArcGIS\Projects\label_sar\label_sar.gdb\TempRaster'
#         out_path = rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_barents\{f_name}'
#         stride = tile_size / 2
#         print(f'Processing: {out_path}')

#         try:
#         # arcpy.env.workspace = r'OK'
#         ##Copy File RasterDataset to GDB Dataset with Background and Nodata setting
#             arcpy.management.CopyRaster(
#                 in_raster = raster_path,
#                 out_rasterdataset = temp_8bit_path,
#                 # nodata_value = '0',
#                 pixel_type = '8_BIT_UNSIGNED',
#                 scale_pixel_value = 'NONE'
#                 )
#         except:
#             print("Copy Raster example failed.")
#             print(arcpy.GetMessages())
    
#         # Execute the ExportTrainingDataForDeepLearning tool
#         print('exporting training samples...')
#         ExportTrainingDataForDeepLearning(
#             in_raster=temp_8bit_path, #temp_raster_path
#             out_folder=out_path,
#             in_class_data=shp_path,
#             image_chip_format='TIFF',
#             tile_size_x=f'{tile_size}',
#             tile_size_y=f'{tile_size}',
#             stride_x=stride,
#             stride_y=stride,
#             metadata_format='Classified_Tiles',
#             buffer_radius=0,
#             # rotation_angle=90,
#             min_polygon_overlap_ratio=0.15, #5min47 without this #538 with
#         )
#         # Delete temp 8bit raster
#         arcpy.management.Delete(temp_8bit_path)
        
#     print(f'Processed month {folder}') 

### Rune North Sea

In [ ]:
# # from arcpy import env
# # arcpy.env.parallelProcessingFactor = "50%"

# # Loop through each folder
# for folder in os.listdir(r"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_rune_shp"):
#     # Define the path to the shp folders
#     month_folder = rf"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_rune_shp\{folder}"

#     # Find shapefile and tif files paths
#     for tif_name in os.listdir(month_folder):
#         f_name = tif_name.strip('.tiff')
#         shp_path = os.path.join(month_folder, f'{tif_name}/{f_name}.shp')
#         # print(shp_path)

#         raster_path = rf"C:\Users\{user_folder}\.1\--Data\s1\rune_reproj_{pixel_size}\{f_name}.tif"
#         temp_8bit_path = r'C:\Users\{user_folder}\OneDrive - University of Bergen\ArcGIS\Projects\label_sar\label_sar.gdb\TempRaster'
#         out_path = rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_rune\{f_name}'
#         stride = tile_size / 2
#         print(f'Processing: {out_path}')

#         try:
#         # arcpy.env.workspace = r'OK'
#         ##Copy File RasterDataset to GDB Dataset with Background and Nodata setting
#             arcpy.management.CopyRaster(
#                 in_raster = raster_path,
#                 out_rasterdataset = temp_8bit_path,
#                 # nodata_value = '0',
#                 pixel_type = '8_BIT_UNSIGNED',
#                 scale_pixel_value = 'NONE'
#                 )
#         except:
#             print("Copy Raster example failed.")
#             print(arcpy.GetMessages())
    
#         # Execute the ExportTrainingDataForDeepLearning tool
#         print('exporting training samples...')
#         ExportTrainingDataForDeepLearning(
#             in_raster=temp_8bit_path, #temp_raster_path
#             out_folder=out_path,
#             in_class_data=shp_path,
#             image_chip_format='TIFF',
#             tile_size_x=f'{tile_size}',
#             tile_size_y=f'{tile_size}',
#             stride_x=stride,
#             stride_y=stride,
#             metadata_format='Classified_Tiles',
#             buffer_radius=0,
#             # rotation_angle=90,
#             min_polygon_overlap_ratio=0.15, #5min47 without this #538 with
#         )
#         # Delete temp 8bit raster
#         arcpy.management.Delete(temp_8bit_path)
        
#     print(f'Processed month {folder}') 

### North Sea 2019

In [ ]:
# # from arcpy import env
# # arcpy.env.parallelProcessingFactor = "50%"

# # Loop through each folder
# for folder in os.listdir(r"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_ns2019_shp"):
#     # Define the path to the shp folders
#     month_folder = rf"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_ns2019_shp\{folder}"

#     # Find shapefile and tif files paths
#     for tif_name in os.listdir(month_folder):
#         f_name = tif_name.strip('.tiff')
#         shp_path = os.path.join(month_folder, f'{tif_name}/{f_name}.shp')
#         # print(shp_path)

#         raster_path = rf"C:\Users\{user_folder}\.1\--Data\s1\ns2019_reproj_{pixel_size}\{f_name}.tif"
#         temp_8bit_path = r'C:\Users\{user_folder}\OneDrive - University of Bergen\ArcGIS\Projects\label_sar\label_sar.gdb\TempRaster'
#         out_path = rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_ns2019\{f_name}'
#         stride = tile_size / 2
#         print(f'Processing: {out_path}')

#         try:
#         # arcpy.env.workspace = r'OK'
#         ##Copy File RasterDataset to GDB Dataset with Background and Nodata setting
#             arcpy.management.CopyRaster(
#                 in_raster = raster_path,
#                 out_rasterdataset = temp_8bit_path,
#                 # nodata_value = '0',
#                 pixel_type = '8_BIT_UNSIGNED',
#                 scale_pixel_value = 'NONE'
#                 )
#         except:
#             print("Copy Raster example failed.")
#             print(arcpy.GetMessages())
    
#         # Execute the ExportTrainingDataForDeepLearning tool
#         print('exporting training samples...')
#         ExportTrainingDataForDeepLearning(
#             in_raster=temp_8bit_path, #temp_raster_path
#             out_folder=out_path,
#             in_class_data=shp_path,
#             image_chip_format='TIFF',
#             tile_size_x=f'{tile_size}',
#             tile_size_y=f'{tile_size}',
#             stride_x=stride,
#             stride_y=stride,
#             metadata_format='Classified_Tiles',
#             buffer_radius=0,
#             # rotation_angle=90,
#             min_polygon_overlap_ratio=0.15, #5min47 without this #538 with
#         )
#         # Delete temp 8bit raster
#         arcpy.management.Delete(temp_8bit_path)
        
#     print(f'Processed month {folder}') 

## 512 x 10

In [ ]:
tile_size = 512
pixel_size = 10

### PKF

In [ ]:
# # from arcpy import env
# # arcpy.env.parallelProcessingFactor = "50%"

# # Loop through each folder
# for folder in os.listdir(r"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_pkf_shp"):
#     # Define the path to the shp folders
#     month_folder = rf"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_pkf_shp\{folder}"

#     # Find shapefile and tif files paths
#     for tif_name in os.listdir(month_folder):
#         f_name = tif_name.strip('.tiff')
#         shp_path = os.path.join(month_folder, f'{tif_name}/{f_name}.shp')
#         # print(shp_path)

#         raster_path = rf"C:\Users\{user_folder}\.1\--Data\s1\pkf_reproj_{pixel_size}\{f_name}.tif"
#         temp_8bit_path = r'C:\Users\{user_folder}\OneDrive - University of Bergen\ArcGIS\Projects\label_sar\label_sar.gdb\TempRaster'
#         out_path = rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_pkf\{f_name}'
#         stride = tile_size / 2
#         print(f'Processing: {out_path}')

#         try:
#         # arcpy.env.workspace = r'OK'
#         ##Copy File RasterDataset to GDB Dataset with Background and Nodata setting
#             arcpy.management.CopyRaster(
#                 in_raster = raster_path,
#                 out_rasterdataset = temp_8bit_path,
#                 # nodata_value = '0',
#                 pixel_type = '8_BIT_UNSIGNED',
#                 scale_pixel_value = 'NONE'
#                 )
#         except:
#             print("Copy Raster example failed.")
#             print(arcpy.GetMessages())
    
#         # Execute the ExportTrainingDataForDeepLearning tool
#         print('exporting training samples...')
#         ExportTrainingDataForDeepLearning(
#             in_raster=temp_8bit_path, #temp_raster_path
#             out_folder=out_path,
#             in_class_data=shp_path,
#             image_chip_format='TIFF',
#             tile_size_x=f'{tile_size}',
#             tile_size_y=f'{tile_size}',
#             stride_x=stride,
#             stride_y=stride,
#             metadata_format='Classified_Tiles',
#             buffer_radius=0,
#             # rotation_angle=90,
#             min_polygon_overlap_ratio=0.15, #5min47 without this #538 with
#         )
#         # Delete temp 8bit raster
#         arcpy.management.Delete(temp_8bit_path)
        
#     print(f'Processed month {folder}') 

### Barents

In [ ]:
# # from arcpy import env
# # arcpy.env.parallelProcessingFactor = "50%"

# # Loop through each folder
# for folder in os.listdir(r"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_barents_shp"):
#     # Define the path to the shp folders
#     month_folder = rf"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_barents_shp\{folder}"

#     # Find shapefile and tif files paths
#     for tif_name in os.listdir(month_folder):
#         f_name = tif_name.strip('.tiff')
#         shp_path = os.path.join(month_folder, f'{tif_name}/{f_name}.shp')
#         # print(shp_path)

#         raster_path = rf"C:\Users\{user_folder}\.1\--Data\s1\barents_reproj_{pixel_size}\{f_name}.tif"
#         temp_8bit_path = r'C:\Users\{user_folder}\OneDrive - University of Bergen\ArcGIS\Projects\label_sar\label_sar.gdb\TempRaster'
#         out_path = rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_barents\{f_name}'
#         stride = tile_size / 2
#         print(f'Processing: {out_path}')

#         try:
#         # arcpy.env.workspace = r'OK'
#         ##Copy File RasterDataset to GDB Dataset with Background and Nodata setting
#             arcpy.management.CopyRaster(
#                 in_raster = raster_path,
#                 out_rasterdataset = temp_8bit_path,
#                 # nodata_value = '0',
#                 pixel_type = '8_BIT_UNSIGNED',
#                 scale_pixel_value = 'NONE'
#                 )
#         except:
#             print("Copy Raster example failed.")
#             print(arcpy.GetMessages())
    
#         # Execute the ExportTrainingDataForDeepLearning tool
#         print('exporting training samples...')
#         ExportTrainingDataForDeepLearning(
#             in_raster=temp_8bit_path, #temp_raster_path
#             out_folder=out_path,
#             in_class_data=shp_path,
#             image_chip_format='TIFF',
#             tile_size_x=f'{tile_size}',
#             tile_size_y=f'{tile_size}',
#             stride_x=stride,
#             stride_y=stride,
#             metadata_format='Classified_Tiles',
#             buffer_radius=0,
#             # rotation_angle=90,
#             min_polygon_overlap_ratio=0.15, #5min47 without this #538 with
#         )
#         # Delete temp 8bit raster
#         arcpy.management.Delete(temp_8bit_path)
        
#     print(f'Processed month {folder}') 

### Rune North Sea

In [ ]:
# # from arcpy import env
# # arcpy.env.parallelProcessingFactor = "50%"

# # Loop through each folder
# for folder in os.listdir(r"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_rune_shp"):
#     # Define the path to the shp folders
#     month_folder = rf"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_rune_shp\{folder}"

#     # Find shapefile and tif files paths
#     for tif_name in os.listdir(month_folder):
#         f_name = tif_name.strip('.tiff')
#         shp_path = os.path.join(month_folder, f'{tif_name}/{f_name}.shp')
#         # print(shp_path)

#         raster_path = rf"C:\Users\{user_folder}\.1\--Data\s1\rune_reproj_{pixel_size}\{f_name}.tif"
#         temp_8bit_path = r'C:\Users\{user_folder}\OneDrive - University of Bergen\ArcGIS\Projects\label_sar\label_sar.gdb\TempRaster'
#         out_path = rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_rune\{f_name}'
#         stride = tile_size / 2
#         print(f'Processing: {out_path}')

#         try:
#         # arcpy.env.workspace = r'OK'
#         ##Copy File RasterDataset to GDB Dataset with Background and Nodata setting
#             arcpy.management.CopyRaster(
#                 in_raster = raster_path,
#                 out_rasterdataset = temp_8bit_path,
#                 # nodata_value = '0',
#                 pixel_type = '8_BIT_UNSIGNED',
#                 scale_pixel_value = 'NONE'
#                 )
#         except:
#             print("Copy Raster example failed.")
#             print(arcpy.GetMessages())
    
#         # Execute the ExportTrainingDataForDeepLearning tool
#         print('exporting training samples...')
#         ExportTrainingDataForDeepLearning(
#             in_raster=temp_8bit_path, #temp_raster_path
#             out_folder=out_path,
#             in_class_data=shp_path,
#             image_chip_format='TIFF',
#             tile_size_x=f'{tile_size}',
#             tile_size_y=f'{tile_size}',
#             stride_x=stride,
#             stride_y=stride,
#             metadata_format='Classified_Tiles',
#             buffer_radius=0,
#             # rotation_angle=90,
#             min_polygon_overlap_ratio=0.15, #5min47 without this #538 with
#         )
#         # Delete temp 8bit raster
#         arcpy.management.Delete(temp_8bit_path)
        
#     print(f'Processed month {folder}') 

### North Sea 2019

In [ ]:
# # from arcpy import env
# # arcpy.env.parallelProcessingFactor = "50%"

# # Loop through each folder
# for folder in os.listdir(r"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_ns2019_shp"):
#     # Define the path to the shp folders
#     month_folder = rf"C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1_ns2019_shp\{folder}"

#     # Find shapefile and tif files paths
#     for tif_name in os.listdir(month_folder):
#         f_name = tif_name.strip('.tiff')
#         shp_path = os.path.join(month_folder, f'{tif_name}/{f_name}.shp')
#         # print(shp_path)

#         raster_path = rf"C:\Users\{user_folder}\.1\--Data\s1\ns2019_reproj_{pixel_size}\{f_name}.tif"
#         temp_8bit_path = r'C:\Users\{user_folder}\OneDrive - University of Bergen\ArcGIS\Projects\label_sar\label_sar.gdb\TempRaster'
#         out_path = rf'C:\Users\{user_folder}\.1\--Data\training_samples\ts_{tile_size}_{pixel_size}\ts_ns2019\{f_name}'
#         stride = tile_size / 2
#         print(f'Processing: {out_path}')

#         try:
#         # arcpy.env.workspace = r'OK'
#         ##Copy File RasterDataset to GDB Dataset with Background and Nodata setting
#             arcpy.management.CopyRaster(
#                 in_raster = raster_path,
#                 out_rasterdataset = temp_8bit_path,
#                 # nodata_value = '0',
#                 pixel_type = '8_BIT_UNSIGNED',
#                 scale_pixel_value = 'NONE'
#                 )
#         except:
#             print("Copy Raster example failed.")
#             print(arcpy.GetMessages())
    
#         # Execute the ExportTrainingDataForDeepLearning tool
#         print('exporting training samples...')
#         ExportTrainingDataForDeepLearning(
#             in_raster=temp_8bit_path, #temp_raster_path
#             out_folder=out_path,
#             in_class_data=shp_path,
#             image_chip_format='TIFF',
#             tile_size_x=f'{tile_size}',
#             tile_size_y=f'{tile_size}',
#             stride_x=stride,
#             stride_y=stride,
#             metadata_format='Classified_Tiles',
#             buffer_radius=0,
#             # rotation_angle=90,
#             min_polygon_overlap_ratio=0.15, #5min47 without this #538 with
#         )
#         # Delete temp 8bit raster
#         arcpy.management.Delete(temp_8bit_path)
        
#     print(f'Processed month {folder}') 

In [ ]:
# Mamen j'espère que ça a passé
break

# Normalize training samples (fix)

Fix script

In [ ]:
# import rasterio
# import os
# import numpy as np

# # Function to normalize image pixel values between 0 and 1
# def normalize_image(image_data):
#     # Normalize to the range [0, 1]
#     min_val = np.min(image_data)
#     max_val = np.max(image_data)
#     return (image_data - min_val) / (max_val - min_val)
#  # # Normalize raster values (0-1)
#         # # Get the minimum and maximum values of the raster
#         # raster_min = arcpy.GetRasterProperties_management(reproj_raster_path, "MINIMUM")
#         # raster_max = arcpy.GetRasterProperties_management(reproj_raster_path, "MAXIMUM")
#         # # Ensure float min max values
#         # raster_min_value = float(raster_min.getOutput(0))
#         # raster_max_value = float(raster_max.getOutput(0))

#         # # Normalize equation
#         # normalized_raster = (Raster(reproj_raster_path) - raster_min_value) / (raster_max_value - raster_min_value)
        
# # Path to the root folder containing subfolders
# root_folder = r"C:\Users\{user_folder}\.1\--Data\ts_512_10\ts_rune"

# # Iterate through the subfolders in the root folder
# for folder in os.listdir(root_folder):
#     folder_path = os.path.join(root_folder, folder)

#     # Check if it's a directory (subfolder)
#     if os.path.isdir(folder_path):
#         images_folder_path = os.path.join(folder_path, 'images')
        
#         # Check if 'images' subfolder exists
#         if os.path.exists(images_folder_path):
#             # Iterate through all files in the 'images' subfolder
#             for file_name in os.listdir(images_folder_path):
#                 file_path = os.path.join(images_folder_path, file_name)

#                 # Only process .tif files
#                 if file_name.endswith('.tif'):
#                     # Open the .tif file using rasterio
#                     with rasterio.open(file_path, 'r+') as src:
#                         image_data = src.read(1)  # Read the first band (assuming single band, modify if multi-band)
                        
#                         # Normalize the pixel values
#                         normalized_data = normalize_image(image_data)

#                         # Overwrite the existing .tif file with normalized data
#                         src.write(normalized_data, 1)  # Write the normalized data to the first band
#                         print(f"Normalized and overwritten: {file_path}")



# Varia

## Count images and slicks

In [ ]:
import geopandas as gpd

gdf = gpd.read_file(r'C:\{user_folder}\.1\--Data\labeled_slick_polygons\s1iwvv_ns2019.geojson')
gdf.drop(columns={'Shape_Length','Shape_Area','OBJECTID'},inplace=True,errors='ignore')
print(gdf['date'].nunique())
print(len(gdf))

361
1138


In [ ]:
import geopandas as gpd

gdf = gpd.read_file(r'C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1iwvv_rune.geojson')
gdf.drop(columns={'Shape_Length','Shape_Area','OBJECTID'},inplace=True,errors='ignore')
print(gdf['date'].nunique())
print(len(gdf))


102
585


In [ ]:
import geopandas as gpd

gdf = gpd.read_file(r'C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1ewhh_pkf.geojson')
gdf.drop(columns={'Shape_Length','Shape_Area','OBJECTID'},inplace=True,errors='ignore')
print(gdf['date'].nunique())
print(len(gdf))


70
81


In [ ]:
import geopandas as gpd

gdf = gpd.read_file(r'C:\Users\{user_folder}\.1\--Data\labeled_slick_polygons\s1ewhh_barents.geojson')
gdf.drop(columns={'Shape_Length','Shape_Area','OBJECTID'},inplace=True,errors='ignore')
print(gdf['date'].nunique())
print(len(gdf))


88
283


In [ ]:
gdf

,date,geometry
0,2019_01_02_170907.tiff,"POLYGON ((7.50694 57.18104, 7.52037 57.1787, 7..."
1,2019_01_02_170907.tiff,"POLYGON ((7.58443 57.21511, 7.58871 57.21336, ..."
2,2019_01_02_170932.tiff,"POLYGON ((9.4657 58.66601, 9.47829 58.68432, 9..."
3,2019_01_03_054830.tiff,"POLYGON ((8.44138 57.6134, 8.45048 57.60593, 8..."
4,2019_01_04_174234.tiff,"POLYGON ((-0.98725 55.47355, -0.98741 55.47483..."
...,...,...
1133,2019_12_26_172634.tiff,"POLYGON ((4.19972 60.28455, 4.20058 60.28289, ..."
1134,2019_12_26_172634.tiff,"POLYGON ((4.22338 60.27563, 4.22074 60.27378, ..."
1135,2019_12_27_060431.tiff,"POLYGON ((3.03916 60.48378, 3.03769 60.48968, ..."
1136,2019_12_31_062031.tiff,"POLYGON ((-0.14275 58.89698, -0.14235 58.89899..."


In [ ]:
gdf['month'] = gdf['date'].str[5:7]
gdf['month'].value_counts()

month
06    331
05    239
07    169
08    129
09     68
04     45
03     40
10     29
11     29
02     25
01     17
12     17
Name: count, dtype: int64

## Rename test/train samples

In [ ]:
# Path to training samples folder
ts_folder = 'data/training_samples/ts_256_25/ts_svalbard - Copy'

# List all the folders in the ts folder
folders = [f for f in os.listdir(ts_folder) if os.path.isdir(os.path.join(ts_folder, f))]
folders

['sample_1',
 'sample_10',
 'sample_100',
 'sample_101',
 'sample_102',
 'sample_103',
 'sample_104',
 'sample_105',
 'sample_106',
 'sample_107',
 'sample_108',
 'sample_109',
 'sample_11',
 'sample_110',
 'sample_111',
 'sample_112',
 'sample_113',
 'sample_114',
 'sample_115',
 'sample_116',
 'sample_117',
 'sample_118',
 'sample_119',
 'sample_12',
 'sample_120',
 'sample_121',
 'sample_122',
 'sample_123',
 'sample_124',
 'sample_125',
 'sample_126',
 'sample_127',
 'sample_128',
 'sample_129',
 'sample_13',
 'sample_130',
 'sample_131',
 'sample_132',
 'sample_133',
 'sample_134',
 'sample_135',
 'sample_136',
 'sample_137',
 'sample_138',
 'sample_139',
 'sample_14',
 'sample_140',
 'sample_141',
 'sample_142',
 'sample_143',
 'sample_144',
 'sample_145',
 'sample_146',
 'sample_147',
 'sample_148',
 'sample_149',
 'sample_15',
 'sample_150',
 'sample_151',
 'sample_152',
 'sample_153',
 'sample_154',
 'sample_155',
 'sample_156',
 'sample_157',
 'sample_158',
 'sample_16',
 'sa

In [ ]:
# Path to training samples folder
ts_folder = 'data/training_samples/ts_256_25/ts_svalbard - Copy'

# List all the folders in the ts folder
folders = [f for f in os.listdir(ts_folder) if os.path.isdir(os.path.join(ts_folder, f))]

random.seed(1)
rand_ints = random.sample(range(1, 159), 158)

# Rename each folder 
i = 0
for folder in os.listdir(ts_folder):
    old_folder_path = os.path.join(ts_folder, folder)
    new_folder_name = f"image_{rand_ints[i]}"
    new_folder_path = os.path.join(ts_folder, new_folder_name)
    i += 1
    # Rename the folder
    os.rename(old_folder_path, new_folder_path)
    print(f"Renamed '{folder}' to '{new_folder_name}'")

Renamed 'sample_1' to 'image_35'
Renamed 'sample_10' to 'image_146'
Renamed 'sample_100' to 'image_17'
Renamed 'sample_101' to 'image_66'
Renamed 'sample_102' to 'image_31'
Renamed 'sample_103' to 'image_127'
Renamed 'sample_104' to 'image_116'
Renamed 'sample_105' to 'image_121'
Renamed 'sample_106' to 'image_98'
Renamed 'sample_107' to 'image_54'
Renamed 'sample_108' to 'image_25'
Renamed 'sample_109' to 'image_125'
Renamed 'sample_11' to 'image_8'
Renamed 'sample_110' to 'image_100'
Renamed 'sample_111' to 'image_111'
Renamed 'sample_112' to 'image_1'
Renamed 'sample_113' to 'image_115'
Renamed 'sample_114' to 'image_69'
Renamed 'sample_115' to 'image_59'
Renamed 'sample_116' to 'image_27'
Renamed 'sample_117' to 'image_82'
Renamed 'sample_118' to 'image_157'
Renamed 'sample_119' to 'image_6'
Renamed 'sample_12' to 'image_7'
Renamed 'sample_120' to 'image_3'
Renamed 'sample_121' to 'image_150'
Renamed 'sample_122' to 'image_56'
Renamed 'sample_123' to 'image_109'
Renamed 'sample_124